In [1]:
%matplotlib ipympl
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix,tf_cov,assert_cov_positive_definite,corr,print_tensor,print_tensors,constrain_cdf
from models.nn_pdf_common import transform_x,density_estimator,create_pdf_layer_mv,create_partially_monotone_dense_layer,\
    create_monotone_dense_layer,create_positive_weights,create_bias
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from flags import FLAGS
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import os
np.set_printoptions(threshold=np.inf)
from data.registry import mpg
from models.utils import get_train_inputs
from tensorflow.python import debug as tf_debug
import itertools

/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importli

In [2]:
FLAGS.data_set="mpg"
data_loader = mpg() 

loaded data: mpg.npz


In [3]:
class cdf_transform:
    
    def __init__(self, x, y, params, positive_transform):
        self.x_transform = transform_x(params, x)

        self.y1 = tf.slice(y, [0, 0], [-1, 1])

        self.y2 = tf.slice(y, [0, 1], [-1, 1])

        self.xy1 = tf.concat([self.x_transform, self.y1], axis=1)
        self.xy2 = tf.concat([self.x_transform, self.y2], axis=1)

        activation1 = tf.nn.sigmoid
        activation2 = tf.nn.softplus

        arch2 = params['arch2']
        arch3 = params['arch3']
        #     relu_bias_initializer = float(params["relu_bias_initializer"])

        with tf.variable_scope("cdf_transform"):

            with tf.variable_scope("xy1_partially_monotone"):
                self.xy1 = create_partially_monotone_dense_layer(self.xy1, arch2[0], self.x_transform.shape[-1].value,
                                                            1, activation=activation1,
                                                            positive_transform=positive_transform)

            for i, units in enumerate(arch2[1:-1]):
                with tf.variable_scope("xy1_l_%d" % i):
                    self.xy1 = create_monotone_dense_layer(self.xy1, units, activation=activation1,
                                                      positive_transform=positive_transform)

            with tf.variable_scope("xy1_l_%d" % len(arch2)):
                self.xy1_prod = create_monotone_dense_layer(self.xy1, arch2[-1], activation=activation1,
                                                       positive_transform=positive_transform)

            #         with tf.variable_scope("xy1_add_l_%d"%len(arch2)):
            #             xy1_add = create_monotone_dense_layer(xy1, arch2[-1], activation=activation1, positive_transform=positive_transform)

            with tf.variable_scope("xy2_partially_monotone"):
                self.xy2 = create_partially_monotone_dense_layer(self.xy2, arch2[0], self.x_transform.shape[-1].value,
                                                            1, activation=activation1,
                                                            positive_transform=positive_transform)

            for i, units in enumerate(arch2[1:-1]):
                with tf.variable_scope("xy2_l_%d" % i):
                    self.xy2 = create_monotone_dense_layer(self.xy2, units, activation=activation1,
                                                      positive_transform=positive_transform)

            with tf.variable_scope("xy2_l_%d" % len(arch2)):
                self.xy2_prod = create_monotone_dense_layer(self.xy2, arch2[-1], activation=activation1,
                                                       positive_transform=positive_transform)

            #         with tf.variable_scope("xy2_add_l_%d"%len(arch2)):
            #             xy2_add = create_monotone_dense_layer(xy2, arch2[-1], activation=activation1, positive_transform=positive_transform)

            with tf.variable_scope("xy_prod"):
                self.xy_prefinal = tf.multiply(self.xy1_prod, self.xy2_prod)
                #             tf.concat([xy_prefinal, xy1_add,xy2_add], axis=1)
                self.xy_prefinal = create_monotone_dense_layer(self.xy_prefinal, arch3[0], activation=activation2,
                                                          positive_transform=positive_transform)

            for i, units in enumerate(arch3[1:]):
                with tf.variable_scope("xy_prefinal_l_%d" % i):
                    self.xy_prefinal = create_monotone_dense_layer(self.xy_prefinal, units, activation=activation2,
                                                              positive_transform=positive_transform)

            with tf.variable_scope('suspicious'):
                input=self.xy_prefinal
                units=1
                bias_initializer = None
                clip_min=None
                clip_max=None
                self.w = create_positive_weights(shape=[input.shape[-1].value, units], positive_transform=positive_transform)

                if bias_initializer is None:
                    self.b = create_bias(units, clip_max=clip_max,clip_min=clip_min)
                else:
                    self.b = create_bias(initializer=[bias_initializer] * units, clip_max=clip_max,clip_min=clip_min)

                self.res = tf.matmul(input, self.w) + self.b
    #             res = tf.Print(res,[b], message="b", summarize=1000)
    #             res = tf.check_numerics(res, "res")
                self.result = tf.nn.softplus(self.res)
    #             result = tf.Print(result,[result], message="result", summarize=1000)
    #             result = tf.check_numerics(result, "result")
    
def create_pdf_layer(cdf, y):
    with tf.name_scope("pdf_layer"):
        assert cdf.shape[0].value == y.shape[0].value
        assert y.shape[-1].value == 1
        assert cdf.shape[-1].value == 1
        # because it is list and cdf is only 1 element
        return tf.gradients(cdf, y, name="gradient")[0]
    
def create_pdf_layer_mv(cdf, y_components):
    gradients = cdf
    for i, y in enumerate(y_components):
        with tf.name_scope("pdf_layer_mv_%d"%i):
            gradients = create_pdf_layer(gradients, y)
#             gradients = tf.Print(gradients,[gradients], message="gradients_%d :"%i, summarize=1000)
    return gradients

def cdf_transforms(x, y, params, positive_transform, mode):
    with tf.variable_scope("extreme_vals",reuse=tf.AUTO_REUSE):
        y_max_values_vars = tf.get_variable("y_max_values_var", trainable=False, dtype=tf.float32,
                                            shape=(1, y.shape[-1].value), initializer=tf.constant_initializer(np.nan))

        if mode == tf.estimator.ModeKeys.TRAIN:
            y_max_values = tf.reduce_max(y, axis=0, keepdims=True)
            y_max_values_vars = tf.cond(tf.reduce_all(tf.is_nan(y_max_values_vars)), \
                  lambda: tf.assign(y_max_values_vars, y_max_values), \
                  lambda: tf.assign(y_max_values_vars, tf.reduce_max(tf.concat([y_max_values, y_max_values_vars], axis=0),axis=0, keepdims=True)))

    with tf.variable_scope("cdf", reuse=tf.AUTO_REUSE):
        non_normalized_cdf = cdf_transform(x, y, params, positive_transform)
        cdf_normalization = cdf_transform(x, tf.tile(y_max_values_vars, [tf.shape(x)[0], 1]),
                                                       params, positive_transform)
    return non_normalized_cdf, cdf_normalization

In [12]:
tf.reset_default_graph()
tf.set_random_seed(10)

ds = get_train_inputs(data_loader, batch_size=10)
iter = tf.data.Iterator.from_structure(ds.output_types,ds.output_shapes)
training_init_op = iter.make_initializer(ds)
batch=iter.get_next()

global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
y_size = 2   

y=batch['y']
x=batch['x']


# y=tf.Print(y,[tf.train.get_global_step(),y], message="==========================================================start: ", summarize=1)
x_size=1
mode=tf.estimator.ModeKeys.TRAIN
params={}
params['positive_transform'] = "square"
params['learning_rate'] = 0.1
params['arch1'] = []
params['arch2'] = [1]
params['arch3'] = [1]
# params['relu_bias_initializer']=100.0

positive_transform = params['positive_transform']
learning_rate = params["learning_rate"]

if y is None or y.shape[-1].value != 2:
    raise NotImplementedError

y_components = [tf.slice(y, [0,i],[-1,1]) for i in range(y_size)]
y_components_combined = tf.concat(y_components, axis=1)

non_normalized_cdf, cdf_normalization = cdf_transforms(x, y_components_combined, params, positive_transform, mode)

grads = create_pdf_layer_mv(non_normalized_cdf.result, y_components)

log_likelihood = tf.log(tf.maximum(grads,1e-24)) - tf.log(tf.maximum(cdf_normalization.result,1e-24))

loss = tf.negative(tf.reduce_mean(log_likelihood), name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])

train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

# with tf.name_scope("train_op"):
#     trainables = tf.trainable_variables()

#     grads = tf.gradients(loss, trainables)

#     grads_printed = []
#     grads_clipped = []
#     for grad, var in zip(grads, trainables):
# #         print(var.name, var, grad.name, grad)
#         grads_printed.append(tf.Print(grad,[tf.train.get_global_step() ,grad], message="grad_%s: "%var.name, summarize=100))
# #         grads_clipped.append(tf.clip_by_value(grad, -0.0001, 0.0001))
# #         grads_clipped.append(tf.clip_by_norm(grad, 0.01))
    
# #     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  
#     train_op = optimizer.apply_gradients(zip(grads_printed, trainables), global_step=tf.train.get_global_step())

delta=30
y_centre_1 = 0
y_centre_2 = 0
points=100
y_1 = np.linspace(y_centre_1-delta,y_centre_1+delta,points)
y_2 = np.linspace(y_centre_2-delta,y_centre_2+delta,points)

y_grid = np.meshgrid(y_1,y_2)

y_grid_flat = np.concatenate(list(map(lambda a: a.reshape(-1,1), y_grid)), axis=1)
    

def train_loop(iters, tain_op):
    session.run(training_init_op)
    
    print("step, loss: ",session.run([tf.train.get_global_step(),loss]))
  
    for step in range(iters):  
        step_val,loss_val,_=session.run([tf.train.get_global_step(),loss,tain_op])
        if step % 100 ==0:
            print("step: {step}, loss: {loss}".format(step=step_val,loss=loss_val))
        
    print("loss: ",session.run(loss))

# with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.04))) as session:
# #     session = tf_debug.LocalCLIDebugWrapperSession(session)
# #     session.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
#     session.run(tf.global_variables_initializer())  
#     train_loop(10000000, train_op)


In [31]:
session.close()

In [32]:
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.04)))
session.run(tf.global_variables_initializer())  
session.run(training_init_op)
print("step, loss: ",session.run([tf.train.get_global_step(),loss]))
for step in range(514):  
    step_val,loss_val,_=session.run([tf.train.get_global_step(),loss,train_op])
#     if step%100==0:
        
print("step: {step_val}/{step}, loss: {loss}".format(step_val=step_val,step=step,loss=loss_val))
# session.close()

step, loss:  [0, 24.363758]
step: 514/513, loss: 6.285952568054199


In [19]:
grad_params = list(zip(itertools.cycle([loss]), [grads,cdf_normalization.result,cdf_normalization.res, non_normalized_cdf.res] + tf.trainable_variables()))  
grad_params = [(cdf_normalization.result, cdf_normalization.res),(non_normalized_cdf.result, non_normalized_cdf.res)] + grad_params  
grad_vals=session.run([tf.gradients(param1, param2) for param1,param2 in grad_params])    

In [30]:
# for (g1,g2),grad_val in zip(grad_params,grad_vals):
# #     print("name: {var} = {status}".format(var=var.name, status= "NaN" if np.any(np.isnan(grad_val)) else "OK" ))
#     print("name: {var} = {value}".format(var=g2.name, value= grad_val ))
# grad_vals[2]
# session.run(tf.gradients(loss, non_normalized_cdf.res))
# session.run(tf.gradients(loss, grads))
session.run(non_normalized_cdf.result)

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [65]:
session.close()

In [25]:
step_val,loss_val,_=session.run([tf.train.get_global_step(),loss,train_op])

InvalidArgumentError: Not a number (NaN) or infinity (Inf) values detected in gradient. : Tensor had NaN values
	 [[{{node gradients/cdf/cdf_transform/suspicious/CheckNumerics_grad/CheckNumerics}} = CheckNumerics[T=DT_FLOAT, message="Not a number (NaN) or infinity (Inf) values detected in gradient.", _device="/job:localhost/replica:0/task:0/device:GPU:0"](gradients/AddN_20)]]
	 [[{{node Adam/update/_330}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_5431_Adam/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'gradients/cdf/cdf_transform/suspicious/CheckNumerics_grad/CheckNumerics', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File